In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from mne.decoding import CSP
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn import preprocessing
from sklearn.svm import SVC

import numpy as np
import moabb,sys,os
from moabb.datasets import Zhou2016
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank
from pyriemann.estimation import Covariances
import pyriemann
from san_utils import *
from pyriemann.classification import MDM,FgMDM
import warnings
warnings.filterwarnings('ignore')

### FB_MDM  ###





In [ ]:
dataset = Zhou2016()
dataset_name="Zhou2016"
allfilters=[[8, 12], [12, 16], [16, 20], [20, 24], [24, 28], [28, 34],[34,38]]
subjects = [1, 2, 3,4]; n_class=3

paradigm = MotorImagery(n_classes=n_class,fmin=8, fmax=30)
X, y, meta = paradigm.get_data(dataset=dataset, subjects=[1])
le,le_label_map=label_map_fn(y)
nchan=X.shape[1]
nfilt=len(allfilters)

pipelines = {}
fb1 = FilterBank(make_pipeline(Covariances(estimator="oas"), CSP(nfilter=4)))
fb2 = FilterBank(make_pipeline(Covariances(estimator="oas"), TangentSpace()))

pipelines["FBCSP+LDA"] = make_pipeline(fb1, LDA())
pipelines["FBCSP+SVM"] = make_pipeline(fb1, SVC(kernel="linear"))
pipelines["FBTS+LDA"] = make_pipeline(fb2, LDA())
pipelines["FBTS+SVM"] = make_pipeline(fb2, SVC(kernel="linear"))


In [ ]:
paradigm = FilterBankMotorImagery(filters=allfilters,n_classes=n_class)
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline',
                           'acc','recall','precision','f1','kappa', 'sensitivity', 'specificity'])

for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])
    ylab = le.transform(y)
    select_indices = list(np.where(meta["session"] == "session_0")[0])
    X0=X[select_indices,:];y0=ylab[select_indices]
    select_indices = list(np.where(meta["session"] == "session_1")[0])
    X1=X[select_indices,:];y1=ylab[select_indices]
    select_indices = list(np.where(meta["session"] == "session_2")[0])
    X2=X[select_indices,:];y2=ylab[select_indices]
    
    #### FB CSP,TS classifiers ###
    for pipeline in pipelines:
        pipe=pipelines[pipeline]
        pipe.fit(X0, y0)
        yp1=pipe.predict(X1); yp2=pipe.predict(X2); 
        acc1,recall1,precision1,f11,kappa1,sens1,spec1=all_metrics(y1,yp1)
        acc2,recall2,precision2,f12,kappa2,sens2,spec2=all_metrics(y2,yp2)
        acc=(acc1+acc2)/2;recall=(recall1+recall2)/2;precision=(precision1+precision2)/2;f1=(f11+f12)/2;
        kappa=(kappa1+kappa2)/2;sens=(sens1+sens2)/2;spec=(spec1+spec2)/2;
        df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, pipeline, acc,recall,precision,f1,kappa,sens,spec]
    #### FB_MDM ###
    fb1 = FilterBank(make_pipeline(Covariances(estimator="oas"),MDM()), flatten=False)
    a,yp1=FB_MDM(X0, y0,X1,y1,fb1);a,yp2=FB_MDM(X0, y0,X2,y2,fb1)
    acc1,recall1,precision1,f11,kappa1,sens1,spec1=all_metrics(y1,yp1)
    acc2,recall2,precision2,f12,kappa2,sens2,spec2=all_metrics(y2,yp2)
    acc=(acc1+acc2)/2;recall=(recall1+recall2)/2;precision=(precision1+precision2)/2;f1=(f11+f12)/2;
    kappa=(kappa1+kappa2)/2;sens=(sens1+sens2)/2;spec=(spec1+spec2)/2;
    df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, "FB_MDM", acc,recall,precision,f1,kappa,sens,spec]
    #### FB_FgMDM ###
    fb2 = FilterBank(make_pipeline(Covariances(estimator="oas"),FgMDM()), flatten=False)
    a,yp1=FB_FgMDM(X0, y0,X1,y1,fb2);a,yp2=FB_FgMDM(X0, y0,X2,y2,fb2)
    acc1,recall1,precision1,f11,kappa1,sens1,spec1=all_metrics(y1,yp1)
    acc2,recall2,precision2,f12,kappa2,sens2,spec2=all_metrics(y2,yp2)
    acc=(acc1+acc2)/2;recall=(recall1+recall2)/2;precision=(precision1+precision2)/2;f1=(f11+f12)/2;
    kappa=(kappa1+kappa2)/2;sens=(sens1+sens2)/2;spec=(spec1+spec2)/2;
    df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, "FB_FgMDM", acc,recall,precision,f1,kappa,sens,spec]
    

In [ ]:
# saving the results
sfile_name = 'Zhou2016/baseline_fb/Results_multiscale_fixed7bands2.csv'

if(os.path.isfile('./'+sfile_name) ==True):
    print("exists")
    df.to_csv(sfile_name, mode="a", index=False, header=False)
else:
    print("No.. so creating")
    df.to_csv(sfile_name,  index=False, header=True)    


